In [1]:
!nvidia-smi

Thu May 12 12:37:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers==4.18.0
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.2/342.2 KB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.4 MB/s eta 0:00:00


In [ ]:
pip install --upgrade --force-reinstall pyarrow

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
from datasets import load_dataset,Dataset,DatasetDict
import warnings # to ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [5]:
# training data
language = 'hi'
input_file_path = f'../input/squadenhi/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

In [6]:
train.head()

,index,question,context,answer_start,text,c_id
0,57283b4c3acd2414000df76d,जब लंदन यहूदी फोरम स्थापित किया गया था?,"उत्तरी लंदन में स्टैमफोर्ड हिल, स्टैनमोर, गोल्...",546,2006,0
1,5727dbb33acd2414000dee30,लुंडेनविक के एंग्लो-सैक्सन समझौते के पतन के लि...,5 वीं शताब्दी की शुरुआत में रोमन शासन के पतन क...,522,की गिरावट आई से दोहराया वाइकिंग हमलों,1
2,5733ec8ed058e614000b65f0,मानवविज्ञानी आमतौर पर दुनिया को कैसे विभाजित क...,सैद्धांतिक जोर से अपनी परियोजना को विभाजित करन...,96,प्रासंगिक समय अवधि और भौगोलिक क्षेत्रों में वि...,2
3,573405584776f41900661710,पुर्तगाल के पास किस क्षेत्र में लोहे और कोयले ...,पुर्तगाल एक महत्वपूर्ण यूरोपीय खनिज उत्पादक है...,411,उत्तर,3
4,5726c05fdd62a815002e8f65,ट्राइग्लिसराइड में कितने ग्लिसरॉल बैकबोन होते ...,आहार वसा के एक अणु में आमतौर पर कई फैटी एसिड (...,213,एक,4


In [7]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

,index,question,context,answer_start,text,c_id,answers
0,57283b4c3acd2414000df76d,जब लंदन यहूदी फोरम स्थापित किया गया था?,"उत्तरी लंदन में स्टैमफोर्ड हिल, स्टैनमोर, गोल्डर्स ग्रीन, फिंचली, हैम्पस्टेड, हेंडन और एडगवेयर में महत्वपूर्ण यहूदी समुदायों के साथ अधिकांश ब्रिटिश यहूदी लंदन में रहते हैं। लंदन शहर के Bevis मार्क्स आराधनालय लंदन के ऐतिहासिक Sephardic यहूदी समुदाय से सम्बद्ध है। यह यूरोप का एकमात्र ऐसा आराधनालय है जिसने 300 वर्षों से लगातार नियमित सेवाओं का आयोजन किया है। स्टैनमोर और कैनन्स पार्क सिनेगॉग की 1998 में पूरे यूरोप में किसी भी ऑर्थोडॉक्स आराधनालय की सबसे बड़ी सदस्यता है, 1998 में इलफ़र्ड सिनेगॉग (लंदन में भी) से आगे निकल गया। समुदाय ने जवाब में 2006 में लंदन यहूदी फोरम की स्थापना की। विकसित लंदन सरकार के बढ़ते महत्व के लिए।",546,2006,0,"{'answer_start': [546], 'text': ['2006']}"


In [8]:
# Validation data
language = 'hi'
input_file_path = f'../input/squadenhi/squad.translate.dev.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)
dev['answers'] = dev[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
dev.head(1)

,index,question,context,answer_start,text,c_id,answers
0,56e1c9bfe3433e1400423196,बहुपद समय में कमी क्या इसका एक उदाहरण है?,"कमी की अवधारणा का उपयोग करके कई जटिलता वर्गों को परिभाषित किया गया है। एक कमी एक समस्या का दूसरी समस्या में परिवर्तन है। यह एक समस्या की अनौपचारिक धारणा को कम से कम एक और समस्या के रूप में मुश्किल बनाता है। उदाहरण के लिए, यदि कोई समस्या Y के लिए एल्गोरिथ्म का उपयोग करके X को हल किया जा सकता है, तो X, Y से अधिक कठिन नहीं है, और हम कहते हैं कि X, Y को कम कर देता है। कई अलग-अलग प्रकार के कटौती हैं, जिनके आधार पर कटौती की विधि, जैसे कि कुक रिडक्शन, कार्प रिडक्शन और लेविन रिडक्शन, और पोलिनेशन-टाइम रिडक्शन या लॉग-स्पेस रिडक्शन जैसी कटौती की जटिलता पर बाध्य होती है।",378,प्रकार के कटौती,0,"{'answer_start': [378], 'text': ['प्रकार के कटौती']}"


In [9]:
dev.head()

,index,question,context,answer_start,text,c_id,answers
0,56e1c9bfe3433e1400423196,बहुपद समय में कमी क्या इसका एक उदाहरण है?,"कमी की अवधारणा का उपयोग करके कई जटिलता वर्गों को परिभाषित किया गया है। एक कमी एक समस्या का दूसरी समस्या में परिवर्तन है। यह एक समस्या की अनौपचारिक धारणा को कम से कम एक और समस्या के रूप में मुश्किल बनाता है। उदाहरण के लिए, यदि कोई समस्या Y के लिए एल्गोरिथ्म का उपयोग करके X को हल किया जा सकता है, तो X, Y से अधिक कठिन नहीं है, और हम कहते हैं कि X, Y को कम कर देता है। कई अलग-अलग प्रकार के कटौती हैं, जिनके आधार पर कटौती की विधि, जैसे कि कुक रिडक्शन, कार्प रिडक्शन और लेविन रिडक्शन, और पोलिनेशन-टाइम रिडक्शन या लॉग-स्पेस रिडक्शन जैसी कटौती की जटिलता पर बाध्य होती है।",378,प्रकार के कटौती,0,"{'answer_start': [378], 'text': ['प्रकार के कटौती']}"
1,56be8e613aeaaa14008c90d1,सुपर बाउल 50 का विषय क्या था?,"सुपर बाउल 50, 2015 के सीज़न के लिए नेशनल फुटबॉल लीग (एनएफएल) के चैंपियन का निर्धारण करने के लिए एक अमेरिकी फुटबॉल खेल था। अमेरिकी फुटबॉल सम्मेलन (एएफसी) चैंपियन डेनवर ब्रोंकोस ने राष्ट्रीय फुटबॉल सम्मेलन (एनएफसी) चैंपियन कैरोलिना पैंथर्स को 24-10 से हराकर अपना तीसरा सुपर बाउल खिताब जीता। यह खेल 7 फरवरी, 2016 को कैलिफोर्निया के सांता क्लारा में सैन फ्रांसिस्को बे एरिया के लेवी स्टेडियम में खेला गया था। चूंकि यह 50 वां सुपर बाउल था, इसलिए लीग ने विभिन्न स्वर्ण-थीम वाली पहलों के साथ ""गोल्डन एनिवर्सरी"" पर जोर दिया, साथ ही रोमन सुपर के साथ प्रत्येक सुपर बाउल गेम के नामकरण की परंपरा को अस्थायी रूप से निलंबित कर दिया (जिसके तहत गेम को ""सुपर बाउल एल"" के रूप में जाना जाता था), ताकि लोगो को अरबी अंकों की संख्या 50 की सुविधा मिल सके।",485,"""गोल्डन एनिवर्सरी",1,"{'answer_start': [485], 'text': ['""गोल्डन एनिवर्सरी']}"
2,5726378238643c19005ad315,1967 में सिम्पोजियम में क्या सुझाव दिया गया था,"1965 में, यूके की नेशनल फिजिकल लेबोरेटरी में डोनाल्ड डेविस ने स्वतंत्र रूप से बारान द्वारा विकसित मैसेज रूटिंग पद्धति को विकसित किया। उन्होंने इसे पैकेट स्विचिंग, बारन की तुलना में अधिक सुलभ नाम कहा, और यूके में एक राष्ट्रव्यापी नेटवर्क बनाने का प्रस्ताव दिया। उन्होंने 1966 में प्रस्ताव पर अपनी बात रखी, जिसके बाद रक्षा मंत्रालय (MoD) के एक व्यक्ति ने उन्हें बारां के काम के बारे में बताया। डेविस की टीम (रोजर स्कैंटेबरी) के एक सदस्य ने लॉरेंस रॉबर्ट्स से 1967 एसीएम संगोष्ठी में ऑपरेटिंग सिस्टम सिद्धांत पर मुलाकात की और ने इसे ARPANET में उपयोग के लिए सुझाव दिया।",523,ने इसे ARPANET,2,"{'answer_start': [523], 'text': ['ने इसे ARPANET']}"
3,5711619950c2381900b54ab3,"एक वायुमंडलीय इंजन में, क्या के खिलाफ हवा के दबाव धक्का करता है?","अगला प्रमुख कदम तब हुआ जब जेम्स वाट ने एक अलग कंडेनसर के साथ न्यूकमेन के इंजन का एक उन्नत संस्करण विकसित किया (1763–1775)। बॉल्टन और वाट के शुरुआती इंजनों ने जॉन स्मेटन के न्यूकमेन के उन्नत संस्करण के रूप में आधे कोयले का इस्तेमाल किया। न्यूकमेन और वाट के शुरुआती इंजन ""वायुमंडलीय"" थे। वे वाष्प के विस्तार के दबाव के बजाय भाप को संघनित करके उत्पन्न आंशिक निर्वात में एक पिस्टन धकेलने वाले वायु दबाव से संचालित होते थे। इंजन सिलेंडरों को बड़ा होना था क्योंकि वायुमंडलीय दबाव के कारण उन पर कार्य करने वाला एकमात्र बल था।",367,एक पिस्टन,3,"{'answer_start': [367], 'text': ['एक पिस्टन']}"
4,572689b6dd62a815002e8893,यॉर्कशायर में किसकी ट्रेनें गंतव्यों की सेवा करती हैं?,"ट्रेन ऑपरेटर वर्जिन ट्रेन ईस्ट कोस्ट तीन घंटे की यात्रा के समय के साथ, लंदन किंग्स क्रॉस करने के लिए ट्रेनों की एक आधा प्रति घंटा की आवृत्ति प्रदान करता है, इन सेवाओं के साथ स्कॉटलैंड के लिए डरहम, Darlington, यॉर्क, Doncaster, नेवार्क उत्तरी गेट और पीटरबरो और उत्तर में फोन एडिनबर्ग में कॉल करने वाली सभी ट्रेनें और गाड़ियों की एक छोटी संख्या ग्लासगो, एबरडीन और इनवर्नेस तक बढ़ गई। क्रॉसकाउंट्री ट्रेनें यॉर्कशायर, मिडलैंड्स और दक्षिण पश्चिम में गंतव्यों की सेवा करती हैं। सबसे पहले TransPennine एक्सप्रेस मैनचेस्टर और लिवरपूल के लिए सेवाएं चल रही है। उत्तरी रेल स्थानीय और क्षेत्रीय सेवाएं प्रदान करता है।",382,क्रॉसकाउंट्री,4,"{'answer_start': [382], 'text': ['क्रॉसकाउंट्री']}"


In [10]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(dev)


squad = DatasetDict()

squad['train'] = tds
squad['validation'] = vds

In [11]:
squad

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 85804
    })
    validation: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 34111
    })
})

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

pretraining_language = 'hi'
# model = AutoModelForQuestionAnswering.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384")
model = AutoModelForQuestionAnswering.from_pretrained(f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450M [00:00<?, ?B/s]

Some weights of the model checkpoint at subhasisj/hi-TAPT-MLM-MiniLM were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at subhasisj/hi-TAPT-MLM-MiniLM and are

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [13]:
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["index"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [14]:
batch_size = 32

In [15]:
squad = squad.map(prepare_train_features,batched=True,remove_columns=squad["train"].column_names)

  0%|          | 0/86 [00:00<?, ?ba/s]

  0%|          | 0/35 [00:00<?, ?ba/s]

In [16]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 3e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    warmup_ratio = 0.1,
    gradient_accumulation_steps = 8,
    num_train_epochs = 5,
    fp16=True,
    weight_decay = 0.01,
#     push_to_hub=True,
    save_total_limit=1,
    load_best_model_at_end=True
)

In [17]:
from transformers import default_data_collator

data_collator = default_data_collator

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=squad["train"],
    eval_dataset=squad["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using amp half precision backend


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
trainer.train()

***** Running training *****
  Num examples = 91554
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 1785
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
0,No log,2.488162
1,3.247600,1.928413
2,1.883100,1.747219
3,1.883100,1.686448
4,1.565400,1.677973


***** Running Evaluation *****
  Num examples = 37015
  Batch size = 32
Saving model checkpoint to ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-357
Configuration saved in ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-357/config.json
Model weights saved in ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-357/pytorch_model.bin
tokenizer config file saved in ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-357/tokenizer_config.json
Special tokens file saved in ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-357/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 37015
  Batch size = 32
Saving model checkpoint to ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-714
Configuration saved in ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-714/config.json
Model weights saved in ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-714/pytorch_model.bin
tokenizer config file saved in ./hi-finetuned-squad-qa-minilmv2-32/checkpoint-714/tokenizer_config.json
Special tokens file saved in ./h

TrainOutput(global_step=1785, training_loss=2.110068766276042, metrics={'train_runtime': 9587.4956, 'train_samples_per_second': 47.747, 'train_steps_per_second': 0.186, 'total_flos': 2.245204423629619e+16, 'train_loss': 2.110068766276042, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub("Training Completed.")

In [ ]:
validation_features = squad["validation"]

In [ ]:
# validation_features = squad_valid.map(
#     prepare_validation_features,
#     batched=True,
#     remove_columns=squad_valid.column_names
# )

In [20]:
!tar -zcvf hi-finetuned-squad-qa-minilmv2-32.tar.gz /kaggle/working/hi-finetuned-squad-qa-minilmv2-32/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
tar: Removing leading `/' from member names
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/checkpoint-1785/
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/checkpoint-1785/sentencepiece.bpe.model
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/checkpoint-1785/training_args.bin
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/checkpoint-1785/pytorch_model.bin
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/checkpoint-1785/config.json
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/checkpoint-1785/tokenizer.json
/kaggle/working/hi-finetuned-squad-qa-minilmv2-32/checkpoint-1785/special_tokens_map.json
/kaggle

In [21]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink

FileLink(r'./hi-finetuned-squad-qa-minilmv2-32.tar.gz')

/kaggle/working/hi-finetuned-squad-qa-minilmv2-32.tar.gz

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/pytorch_model.bin"')

In [ ]:
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/config.json"')

In [ ]:
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/tokenizer_config.json"')

In [ ]:
FileLink(f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}/special_tokens_map.json"')

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
import collections

examples = squad_valid
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
            # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(squad_valid, validation_features, raw_predictions.predictions)


In [ ]:
from datasets import load_metric
metric = load_metric("squad")

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad_valid]
metric.compute(predictions=formatted_predictions, references=references)